# Google ADK Crash Course · Module 2: Introduction to ADK & Comparisons

In this module, you will:

1. Understand what **Agent Development Kit (ADK)** is and why it exists.
2. Learn the **core building blocks** (Agents, Tools, Workflows, Memory, Callbacks, Dev UI/CLI).
3. See the **architecture & mental model** (root agent, sub-agents, delegation).
4. Explore **workflow patterns** (sequential, parallel, loop).
5. Compare ADK with **LangChain/LangGraph**, **CrewAI**, and **Microsoft AutoGen**.
6. Note **strengths, limitations, and gotchas** so you can pick the right tool for the job.

> This notebook is conceptual (heavy on explanations) with a few small code sanity checks and pattern snippets. Use it to teach or to orient your team before coding.

## What is ADK (Agent Development Kit)?

**ADK** is a flexible, modular framework for building and deploying **AI agents**, optimized for **Gemini** but **model-agnostic** (you can also use GPT/Claude) and **deployment-agnostic** (local or cloud). It aims to make agent development feel like standard software engineering: clear configs, testable units, composable workflows, and an integrated developer experience (CLI + web UI).

**Key ideas**:
- **Agents** are self-contained execution units that pursue goals, call tools, and can coordinate with other agents.
- **Tools** give agents capabilities (search, code execution, RAG, custom Python functions, third-party APIs).
- **Workflows** orchestrate agents (e.g., run in sequence, in parallel, or iterate in loops).
- **Memory/Session State** lets agents carry context across steps/sessions.
- **Callbacks/Guardrails** hook into lifecycle stages for safety, logging, and control.
- **Dev Experience** includes an `adk` CLI and an interactive web UI for local testing.

We’ll dive deeper into each of these in the next sections and hands-on modules.

## Core Concepts: Agents, Tools, Workflows, Memory, Callbacks

### Agents
- An **Agent** is a goal-driven unit that can reason, call tools, and produce outputs.
- A **root agent** is your entry point. It can delegate work to **sub-agents** specialized for tasks (research, coding, analysis, etc.).
- Multi-agent systems in ADK are built by composing multiple agents with defined responsibilities.

### Tools
- **Built-in tools** (e.g., Google Search, code executors, RAG) enable common capabilities out-of-the-box.
- **Function tools** are custom Python functions you expose to agents.
- **Third-party tools** or even **agents-as-tools** can be integrated for complex workloads.

### Workflows
- **Sequential**: sub-agents run in a fixed order (A → B → C).
- **Parallel**: sub-agents run concurrently and aggregate results.
- **Loop**: iterate until a condition or iteration budget is met (e.g., refine a draft until it passes checks).

### Memory & Session State
- Persist useful state across turns or app restarts (e.g., user profile, intermediate artifacts).

### Callbacks / Guardrails
- Hook into agent lifecycle (before/after tool calls, on error) to enforce constraints, redact data, or log decisions.

## Mental Model & Architecture (ASCII)

```
                    +---------------------+
                    |       Client        |
                    |  (UI, API, tests)  |
                    +----------+----------+
                               |
                          (Request)
                               v
                        +-------------+
                        |  RootAgent  |  <— entry point
                        +------+------+     - routing/delegation
                               |
     +-------------------------+--------------------------+
     v                                                    v
+---------+      tools      +-----------+         +--------------+
| Search  | <-------------- |  Research |         |   Analyzer   |
|  Tool   |                 |  Agent    |         |   Agent      |
+---------+                 +-----------+         +--------------+
     |                            |                        |
     |                            v                        v
  (web)                      +----------+            +-----------+
                             |  Memory  |            |  Tools    |
                             +----------+            +-----------+

Workflow agents:
- SequentialAgent: Research → Analyzer → Reporter
- ParallelAgent:   Analyzer(CPU) || Analyzer(GPU) || Analyzer(API)
- LoopAgent:       Iterate(Analyze → Improve) until done
```

## ADK vs Other Frameworks (Quick Comparison)

| Feature / Concern | **ADK** | **LangChain / LangGraph** | **CrewAI** | **Microsoft AutoGen** |
|---|---|---|---|---|
| Core focus | Modular agents, tools, **workflows**; strong dev UX (CLI/UI) | General LLM app framework; **LangGraph** for durable, controllable agents | Role-based multi-agent "crews" + **Flows** for deterministic orchestration | Event-driven multi-agent conversations; humans/tools/LLMs in the loop |
| Built-in tools | Google Search, code exec, RAG, etc. (Gemini-focused) | Many integrations via toolkits; now nudging toward LangGraph for agents | Rich agent roles/tools; expanding code-exec features | Agents, tools, human-in-the-loop; code agents; AutoGen Studio (UI) |
| Models | Gemini-first, but **model-agnostic** (GPT/Claude supported) | Open, many providers; strong ecosystem | Open; supports multiple providers | Open; supports multiple providers |
| Workflows | **Sequential, Parallel, Loop** agents first-class | Compose graphs with nodes/edges, retries, persistence | **Crews** (autonomy) & **Flows** (determinism) | Multi-agent chat, tools; custom routing patterns |
| Dev tooling | `adk` CLI, local web UI; samples/codelabs | LangSmith/LangGraph Platform, tracing, eval | CLI + templates; community courses; quickstart | SDK + examples; AutoGen Studio research tool |
| Productionization | Vertex AI alignment; local-to-cloud paths | LangGraph deploy, platform features | Python-first; community driven | Python-first; event-driven scaling |
| Where it shines | Opinionated **agent workflows**, built-ins, DX | Huge ecosystem + graph control | Team-of-agents persona workflows | Conversational multi-agent protocols |
| Watch-outs | Some built-ins are **Gemini-only** and may be **root-agent only** | Rapidly evolving APIs (LangGraph migration) | Autonomy can be harder to constrain | Conversation-centric design requires orchestration care |

> Use this table to position your project. If you need first-class **workflow agents** and an integrated DX, ADK is a great starting point. If you want maximal ecosystem plugins or graph semantics, evaluate LangGraph. If you like persona teams, try CrewAI. If you want event-driven multi-agent chat protocols, try AutoGen.

## Strengths, Limitations & Gotchas (ADK)

**Strengths**
- Clean separation of **agents, tools, and workflows**.
- Built-in **Sequential/Parallel/Loop** workflow agents.
- Solid **dev experience**: CLI, local web UI, samples, codelabs.
- Works beyond Gemini (bring GPT/Claude), while keeping Gemini-first ergonomics.

**Limitations / Gotchas**
- Certain **built-in tools are Gemini-only**; plan non-Gemini fallbacks.
- Some **built-in tools may not be callable from sub-agents**; design tool usage near the root if you hit errors.
- An agent instance is typically added **once** as a sub-agent (clone if needed under different names).
- As with other frameworks, **APIs evolve**. Keep an eye on release notes and `adk --help` output.

We’ll show workarounds (e.g., wrapping capabilities as function tools) in later modules.

## When to Choose Which Framework?

- **Choose ADK** when you want:
  - First-class **workflow agents** (sequential/parallel/loop) and a cohesive DX.
  - Strong Gemini integration but flexibility to use non-Gemini models.
  - A clean separation between agents, tools, and orchestration.

- **Choose LangChain/LangGraph** when you want:
  - Maximum ecosystem integrations and **graph semantics** (stateful nodes, retries, persistence).
  - LangSmith tracing/evals and platform deployment options.

- **Choose CrewAI** when you want:
  - Persona-driven **crews** collaborating autonomously, or **Flows** for deterministic control.

- **Choose AutoGen** when you want:
  - Event-driven **multi-agent conversations** (agents + humans + tools) as the primary abstraction.

You can also **mix and match**: wrap an ADK agent as a function/tool inside another framework, or vice versa, if a particular capability is missing.